In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
file_path = '/mnt/data/Iris.csv'
iris_data = pd.read_csv(file_path)

# Prepare the data
X = iris_data.drop(columns=['Id', 'Species'])
y = iris_data['Species']

# Encode the class labels as integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Define the structure of the tree node
class TreeNode:
    def __init__(self, gini, num_samples, num_samples_per_class, predicted_class):
        self.gini = gini
        self.num_samples = num_samples
        self.num_samples_per_class = num_samples_per_class
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None

# Function to calculate the Gini index
def gini_index(y):
    m = len(y)
    return 1.0 - sum((np.sum(y == c) / m) ** 2 for c in np.unique(y))

# Function to find the best split
def best_split(X, y):
    m, n = X.shape
    if m <= 1:
        return None, None

    num_classes = len(np.unique(y))
    num_parent = [np.sum(y == c) for c in range(num_classes)]
    best_gini = 1.0 - sum((num / m) ** 2 for num in num_parent)
    best_idx, best_thr = None, None

    for idx in range(n):
        thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
        num_left = [0] * num_classes
        num_right = num_parent.copy()
        for i in range(1, m):
            c = classes[i - 1]
            num_left[c] += 1
            num_right[c] -= 1
            gini_left = 1.0 - sum((num_left[x] / i) ** 2 for x in range(num_classes))
            gini_right = 1.0 - sum((num_right[x] / (m - i)) ** 2 for x in range(num_classes))
            gini = (i * gini_left + (m - i) * gini_right) / m
            if thresholds[i] == thresholds[i - 1]:
                continue
            if gini < best_gini:
                best_gini = gini
                best_idx = idx
                best_thr = (thresholds[i] + thresholds[i - 1]) / 2
    return best_idx, best_thr

# Recursive function to build the tree
def build_tree(X, y, depth=0, max_depth=10):
    num_samples_per_class = [np.sum(y == i) for i in range(len(np.unique(y)))]
    predicted_class = np.argmax(num_samples_per_class)
    node = TreeNode(
        gini=gini_index(y),
        num_samples=len(y),
        num_samples_per_class=num_samples_per_class,
        predicted_class=predicted_class,
    )

    if depth < max_depth:
        idx, thr = best_split(X, y)
        if idx is not None:
            indices_left = X[:, idx] < thr
            X_left, y_left = X[indices_left], y[indices_left]
            X_right, y_right = X[~indices_left], y[~indices_left]
            node.feature_index = idx
            node.threshold = thr
            node.left = build_tree(X_left, y_left, depth + 1, max_depth)
            node.right = build_tree(X_right, y_right, depth + 1, max_depth)
    return node

# Train the decision tree
tree = build_tree(X_train.values, y_train)

# Function to print the tree
def print_tree(node, depth=0):
    if node is not None:
        print(f"{'|   ' * depth}Node(depth={depth}, gini={node.gini:.4f}, samples={node.num_samples}, value={node.num_samples_per_class}, class={node.predicted_class})")
        if node.left is not None or node.right is not None:
            print(f"{'|   ' * depth} feature_{node.feature_index} <= {node.threshold:.4f}")
            print_tree(node.left, depth + 1)
            print(f"{'|   ' * depth} feature_{node.feature_index} > {node.threshold:.4f}")
            print_tree(node.right, depth + 1)

# Function to make predictions
def predict_tree(node, X):
    if node.left is None and node.right is None:
        return node.predicted_class
    if X[node.feature_index] <= node.threshold:
        return predict_tree(node.left, X)
    else:
        return predict_tree(node.right, X)

# Print the tree structure
print_tree(tree)

# Make predictions on the test set
y_pred = [predict_tree(tree, x) for x in X_test.values]

# Evaluate the accuracy
accuracy = np.sum(y_pred == y_test) / len(y_test)
print(f"Accuracy: {accuracy:.4f}")
